In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [3]:
with open('truyen_kieu_clean.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print("Vocab size:", vocab_size)
print("Number of characters:", len(text))

Vocab size: 129
Number of characters: 104804


In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Kiểu thơm lần giở trước đèn"))
print(decode(encode("Kiểu thơm lần giở trước đèn")))

[17, 39, 100, 50, 1, 49, 38, 80, 42, 1, 41, 86, 43, 1, 37, 39, 114, 1, 49, 47, 82, 112, 34, 1, 76, 61, 43]
Kiểu thơm lần giở trước đèn


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 2500
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.2

In [7]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
    
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
    
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v 
        return out

class MultiHead(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.multihead = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.multihead], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_heads, n_embd):
        super().__init__()
        self.multihead = MultiHead(n_heads, n_embd//n_heads)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        return

    def forward(self, x):
        x = x + self.multihead(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [8]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_head, n_embd) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.positional_embedding(torch.arange(T, device=device))
        x = tok_emb + pos_emb # B, T, C
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # B, T, vocab_size

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [9]:
model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.9947, val loss 4.9969
step 100: train loss 2.2937, val loss 2.2822
step 200: train loss 2.2094, val loss 2.2043
step 300: train loss 2.1705, val loss 2.1666
step 400: train loss 2.1293, val loss 2.1337
step 500: train loss 2.0439, val loss 2.0530
step 600: train loss 1.9381, val loss 1.9477
step 700: train loss 1.8612, val loss 1.8754
step 800: train loss 1.8019, val loss 1.8192
step 900: train loss 1.7458, val loss 1.7600
step 1000: train loss 1.6989, val loss 1.7241
step 1100: train loss 1.6577, val loss 1.6896
step 1200: train loss 1.6273, val loss 1.6693
step 1300: train loss 1.5926, val loss 1.6449
step 1400: train loss 1.5644, val loss 1.6259
step 1500: train loss 1.5420, val loss 1.6187
step 1600: train loss 1.5156, val loss 1.6097
step 1700: train loss 1.4828, val loss 1.5903
step 1800: train loss 1.4555, val loss 1.5840
step 1900: train loss 1.4279, val loss 1.5753
step 2000: train loss 1.4045, val loss 1.5741
step 2100: train loss 1.3744, val loss 1.5671


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_token=2000)[0].tolist()))


Nành sao lên xuân lại tiền quì,
Thoắt càng trông kéo trả dầu dặm nhg.
Vực tử các đồng đán tòng,
Bấy giờ ngọn bóng như trôn hoa.
Cạn Tương Khí nh nức trăm,
Lòng đàm sắc làm khuyanh Kiều ngày .
Tích nào câu hại ượn như hay,
Họa quì cây lại lầy tia có chăn .
Chư hiều ngảnh ăn Tiểu tróc muông,
Xao thơ một vắng chàng thẹn đêm vàng.
Rằng: Tiếng Vi dặm tội trường,
Thân bàn eo khán quyết nhàng tôi .
Đem thay nước bên trả chăng,
Lện thương cánh cái thừa bên chưa.
Bỗng chiệt thuở lễ đâ,
Đầy thân hỏi dễ bắt ngơ cho gì .
Oan mình bể ta khỏi dàu,
Những gười, xinh vạt bất chuộc va !
Xăm đi chấp càng đá điều,
Ngập thét, ngảng giếp cỏ gì lai.
Nàng nhờn cũng bậm người,
Mà hỏi ê cửa cũng ngừn dài thá cho.
Tường sinh sởngẫm đôn ý nhà,
Hoa rủ dôn cũng hoài hoa nhà nết sầu.
Sinh sinh nành típ áo dồi,
Trên điều đà ngiọt phận kiều hằng !
Từ công, nhẩn đã giữ vỏ tường,
Nàng quen viện tái nên,
Đem tên mio thảng ai đã chi nàng !
Đã giay khi vào,
Cướt triên Thúc liềm họa êm tình.
Nơi suyên tót thiểu thấp dài nh